# Introduction

This code reads the scio and/or saves to a json file

## Needed python libraries
- pySerial

In [ ]:
import json # To read settings & data file
import struct

# Logging/output format setup
import logging
log = logging.getLogger('root')
log.setLevel(logging.DEBUG)
logging.basicConfig(format='[%(asctime)s] %(levelname)8s: %(message)s', datefmt='%Y-%m-%d %H:%M:%S')

In [ ]:
project_path = './'
project_path_output = project_path + '01_rawdata/scan_json/'
# input file (TODO: replace with reading all files in folder)
output_file = 'cal01.json'

# Some general parameters


In [ ]:
# Find SCIO device's serial port
# - - - - - - - - - - - - - - - -
import serial.tools.list_ports as pyserial

def find_scio_dev():
    scio_dev = ''
    scio_ID = '0451:16AA'
    # Find the scio port. Returns an empty list if not found
    scio_port = list(pyserial.grep(scio_ID))
    if(len(scio_port) == 0):
        raise Exception('No SCIO was detected: Is the SCIO on?') 
    scio_dev = scio_port[0].device
    log.info('Using serial port: ' + scio_dev)
    return(scio_dev)

scio_device = find_scio_dev()

In [ ]:
# Supporting functions
#---------------------

def encode_b64(bytestring):
    # Encode and decode data as base64 (to store until we know what to do with it)
    import base64
    urlSafeEncodedBytes = base64.urlsafe_b64encode(bytestring)
    urlSafeEncodedStr = str(urlSafeEncodedBytes, 'utf-8')
    return(urlSafeEncodedStr)

def decode_b64(b64string):
    # Encode and decode data as base64 (to store until we know what to do with it)
    import base64
    bytestring = base64.urlsafe_b64decode(b64string)
    return(bytestring)

def write_raw_file(output_fn, temp_before, temp_after, rawdata):
    # Saves a single scan as raw data to JSON
    from datetime import datetime
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    # create the json file
    jsondata = {}
    jsondata['scan'] = []
    jsondata['scan'].append({
        'timestamp': timestamp,
        't_cmos_before': temp_before[0], # cmosTemperature, chipTemperature, objectTemperature
        't_chip_before': temp_before[1],
        't_obj_before':  temp_before[2],
        't_cmos_after': temp_after[0],
        't_chip_after': temp_after[1],
        't_obj_after':  temp_after[2],
        'part1': encode_b64(rawdata[0]),
        'part2': encode_b64(rawdata[1]),
        'part3': encode_b64(rawdata[2])
    })
    # Write data to JSON file
    import json
    with open(output_fn, 'w') as outfile:
        json.dump(jsondata, outfile, indent=4)

In [ ]:
import serial
from serial.tools import list_ports_common

# Constants
# - - - - - 
READ_DATA = 2        #02
READ_TEMPERATURE = 4 #04
READ_BATTERY = 5     #05
SET_READY = 14       #0e
SET_LED = 11         #0b
PROTOCOL = -70       #ba
READ_BLE_ID = -124
READ_DEV_ID = 1
#RESET_LED = 50 # Does this work?
# The 3 readings are: sample, sampleDark, sampleGradient
# Whitereference is the scan of the box

# Functions
# - - - - -

def decode_temperature(msg, message_length):
    num_vars = message_length / 4 # divide by 4 because we are dealing with longs
    data_struct = '<' + str(int(num_vars)) + 'L' # This is '<3l' or '<lll'
    # Convert bytes to unsigned int
    message_data = struct.unpack(data_struct, msg)
    # Convert to temperatures
    cmosTemperature = (message_data[0] - 375.22) / 1.4092 # Does this make sense? It's from the disassembled Android app...
    chipTemperature = (message_data[1]) / 100
    objectTemperature = message_data[2] / 100
    temperature_df = [cmosTemperature, chipTemperature, objectTemperature]
    return(temperature_df)

def decode_dev_id(msg, message_length):
    print(message_length)
    print(msg)
    print(len(msg))
    #data_struct = '<8s8sH'
    data_struct = '<H'
    dev_df = struct.unpack(data_struct, msg[16:18])
    print(dev_df)
    return(dev_df)

def decode_ble_id(msg, message_length):
    #print(message_length)
    print(msg)
    print(msg[0:8])
    print(msg[8:10]) # BLE firmware version
    print(msg[10:50].decode('utf-8') ) # Unknown ID
    print(msg[50:66].decode('utf-8') ) # device name
    print(msg[66:131].decode('utf-8') ) # i2s tag
    #data_struct = '<8sL16s64s'
    data_struct = '<8sH40s16s64s'
    ble_df = list(struct.unpack(data_struct, msg))
    ble_df[2] = ble_df[2].decode('utf-8').strip()
    ble_df[3] = ble_df[3].decode('utf-8').strip('\x00')
    ble_df[4] = ble_df[4].decode('utf-8').strip()
    data_struct = '<8B'
    ble_df = struct.unpack(data_struct, msg[0:8])
    print(list(map(chr, ble_df)))
    #print(''.join(r'\x%02X' % ord(ch) for ch in msg[0:8] ))
    print(ble_df)
    return(ble_df)

def protocol_message(cmd):
    byte_command = b'' # empty initialisation
    if(cmd == SET_LED):
        # Setting the LED colour is special (longer command)
        byte_command = struct.pack('<bbbbbbbbbbbbbbb',1,PROTOCOL,cmd,9,0,0,0,0,0,0,0,0,0,0)
    else:
        byte_command = struct.pack('<bbbbb',1,PROTOCOL,cmd,0,0)
    return(byte_command)

# Send a command and read the returned data, if the data type is known
def read_scio(scio_dev, command):
    # Send reading command
    # - - - - - - - - - - -
    
    # Create the message
    byte_msg = protocol_message(command)
    
    # Open serial connection
    try:
        ser = serial.Serial(scio_dev)
    except OSError as error:
        log.error(error)
        quit()
    
    # write the message to the serial device
    ser.write(byte_msg)
    
    # Read the response
    # - - - - - - - - - - -
    
    # Start reading the response
    s = ser.read(1)
    message_type    = struct.unpack('<b',s)[0]
    
    # Error check
    assert (message_type == PROTOCOL), 'Wrong message type: {}'.format(message_type)
    
    # Data type
    s = ser.read(1)
    message_content = struct.unpack('<b',s)[0]
    if(message_content == READ_TEMPERATURE):
        log.debug('Receiving temperature data: ' + str(message_content))
        # Data length
        s = ser.read(2)
        message_length = struct.unpack('<H',s)[0]
        # Read the number of values specified by the message length
        s = ser.read(message_length)
        ser.close()
        # decode that data
        df = decode_temperature(s, message_length) # cmosTemperature, chipTemperature, objectTemperature
    elif(message_content == READ_DATA):
        df = [ ]
        log.debug('Receiving scan data: ' + str(message_content))
        # get rid of first 2 sets to get
        for i in range(3):
            log.debug('--> Part: ' + str(i+1))
            if(i > 0):
                s = ser.read(1)
                message_type    = struct.unpack('<b',s)[0]
                s = ser.read(1)
                message_content = struct.unpack('<b',s)[0]
            s = ser.read(2)
            message_length = struct.unpack('<H',s)[0]
            s = ser.read(message_length)
            df.append(s)
        ser.close()
    elif(message_content == READ_DEV_ID):
        s = ser.read(2)
        message_length = struct.unpack('<H',s)[0]
        # Read the number of values specified by the message length
        s = ser.read(message_length)
        ser.close()
        # decode that data
        df = decode_dev_id(s, message_length)
    elif(message_content == READ_BLE_ID):
        s = ser.read(2)
        message_length = struct.unpack('<H',s)[0]
        # Read the number of values specified by the message length
        s = ser.read(message_length)
        ser.close()
        # decode that data
        df = decode_ble_id(s, message_length)
    else:
        log.debug('Receiving unknown message: ' + str(message_content))
    return(df)
        
# Does a full scan including temperature before & after
def scio_scan(scio_serial_port):
    log.info('Checking device temperature...')
    temp_before = read_scio(scio_serial_port, 4) # 4 = read temperature
    log.info('CMOS T: {:.3f}'.format(temp_before[0])) # cmosTemperature, chipTemperature, objectTemperature
    log.info('Chip T: {:.3f}'.format(temp_before[1]))
    log.info('Obj. T: {:.3f}'.format(temp_before[2]))

    # Scan and decode
    log.info('Scanning...')
    scan_raw = read_scio(scio_serial_port, 2) # 2 = read data

    # Read temperature after scanning
    log.info('Checking device temperature again...')
    temp_after = read_scio(scio_serial_port, 4)
    
    df = {}
    df['temp_before'] = temp_before
    df['temp_after']  = temp_after
    df['rawdata'] = scan_raw

    return(df)

In [ ]:
# Do the scan
log.info("Scanning device on serial port: " + scio_device)
data = scio_scan(scio_device)

# Create file name
from datetime import datetime
now = datetime.now()
output_fn = "scio_scan_" + now.strftime("%Y%m%d_%H%M%S") + ".json"

# Save the raw scan data
log.info("Writing to file: " + output_fn)
write_raw_file(project_path_output + output_fn, data['temp_before'], data['temp_after'], data['rawdata'])

## Notes

- The device always transmits the data and the calibration to the SCIO server, as:
  - sample and sample_dark (This starts with base64: AAAAA)
  - sample_white and sample_white_dark (Starts with base64: AAAAA)
  - sample_white_gradient and sample_gradient (Starts with base64: bgAAA)
- Other transmitted data (with examples):
  - "device_id":"8032AB45611198F1"
  - "sampled_at":"2021-10-20T10:58:58.729+03:00"
  - "sampled_white_at":"2021-10-20T10:53:18.334+03:00"
  - "scio_edition":"scio_edition"
  - "mobile_GPS":{"longitude":-----,"latitude":----,"locality":"-----","country":"-----","admin_area":"-----","address_line":"-----" (THIS IS SCARY!)
  - "mobile_mac_address":"------" (SCARY AGAIN!)
  - "i2s_tag_config":"20150812-e:PRODUCTION" (in my case, seems to be a hardware version)
- Data seems encrypted, the CC2540 chip can do AES-128
  - Is the device_id the encryption/decryption key?